# Adding Visual Features

In [ ]:
%load_ext autoreload
%autoreload 2

import os
os.remove('snorkel.db')

import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

from snorkel import SnorkelSession
session = SnorkelSession()

from pdb import set_trace as t

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Convert PDF to HTML

Use Adobe Acrobat (or other program of your choice) to convert PDF -> HTML with structure.

### Parse HTML and PDF

In [ ]:
import os
from snorkel.parser import CorpusParser
from snorkel.parser import HTMLParser
from snorkel.parser import OmniParser

pdf_path = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware100_pdf/'
html_path = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware100_html2/'

filename = 'bc546-d'
html_file = html_path + filename + '.html'

doc_parser = HTMLParser(path=html_file)
context_parser = OmniParser(pdf_path=pdf_path, session=session)
cp = CorpusParser(doc_parser, context_parser, max_docs=1) 

%time corpus = cp.parse_corpus(name='Hardware', session=session)

# Save results
# os.system('cp snorkel.db snorkel.db\ corpus');

Extracted 2474 pdf words
Elapsed: 0.536 s
Extracted 2631 html words
Elapsed: 0.062 s
2474
2474
(54, 76)
[(u"''", u'(BR)CBO'),
 (u"'s", u'(BR)CEO'),
 ('(', u'(BR)EBO'),
 (u'+125', u'(I'),
 (u'-55', u'(MHz'),
 (u'.5', u'(NORMALIZED'),
 (u'1050', u'(Note'),
 (u'2175', u'(Pb\u2212Free'),
 (u'2176', u'(SCILLC)'),
 (u'226', u'(T'),
 (u'25', u'(TO\u2212226'),
 (u'282', u'(V'),
 (u'303', u'(VOLTS'),
 (u'344', u'(mA'),
 (u'3860', u'(mAdc'),
 (u'3867', u'(mV'),
 (u'421\xa033\xa0790\xa02910', u'(pF'),
 (u'55', u'*For'),
 (u'5817', u'+125\xb0C'),
 (u'675', u'-55\xb0C'),
 (u'81', u'/I'),
 (u'92', u'125\xb0C'),
 (u'9855', u'25\xb0C'),
 (u'AYW', u'2910'),
 (u'Adc', u'29\u221211'),
 (u'Ambient', u'303\u2212675\u22122175'),
 (u'BR', u'303\u2212675\u22122176'),
 (u'C/W', u'33'),
 (u'Case', u'421'),
 (u'JA', u'790'),
 (u'JC', u'800\u2212282\u22129855'),
 (u'M', u'800\u2212344\u22123860'),
 (u'Marking.pdf', u'800\u2212344\u22123867'),
 (u'N.', u'81\u22123\u22125817\u22121050'),
 (u'P.O.', u'AYWW'),
 (u'Pb

### Load results

In [ ]:
# import os
# from snorkel.models import Corpus
# from snorkel.utils import get_ORM_instance

# os.system('cp snorkel.db\ corpus snorkel.db');
# corpus = get_ORM_instance(Corpus, session, 'Hardware')

### Extract Candidates

In [ ]:
import os
from collections import defaultdict
from snorkel.models import Corpus, candidate_subclass
from snorkel.matchers import RegexMatchSpan, Union
from snorkel.candidates import CandidateExtractor
from snorkel.utils import get_ORM_instance
from hardware_utils import OmniNgramsPart, OmniNgramsTemp, get_gold_dict

# Candidate Type
Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])
    
# CandidateSpaces
part_ngrams = OmniNgramsPart(parts_by_doc=None, n_max=3) # NOTE: no part linking right now
temp_ngrams = OmniNgramsTemp(n_max=2)

# Matchers
eeca_matcher = RegexMatchSpan(rgx='([b]{1}[abcdefklnpqruyz]{1}[\swxyz]?[0-9]{3,5}[\s]?[A-Z\/]{0,5}[0-9]?[A-Z]?([-][A-Z0-9]{1,7})?([-][A-Z0-9]{1,2})?)')
jedec_matcher = RegexMatchSpan(rgx='([123]N\d{3,4}[A-Z]{0,5}[0-9]?[A-Z]?)')
jis_matcher = RegexMatchSpan(rgx='(2S[abcdefghjkmqrstvz]{1}[\d]{2,4})')
others_matcher = RegexMatchSpan(rgx='((NSVBC|SMBT|MJ|MJE|MPS|MRF|RCA|TIP|ZTX|ZT|TIS|TIPL|DTC|MMBT|PZT){1}[\d]{2,4}[A-Z]{0,3}([-][A-Z0-9]{0,6})?([-][A-Z0-9]{0,1})?)')
parts_matcher = Union(eeca_matcher, jedec_matcher, jis_matcher, others_matcher)

temp_matcher = RegexMatchSpan(rgx=r'[0-9]+[05]', longest_match_only=False)

# Throttler
part_throttler = lambda x: x[0].parent.page == x[1].parent.page

# Extractor
ce = CandidateExtractor(Part_Temp, 
                        [part_ngrams, temp_ngrams], 
                        [parts_matcher, temp_matcher], 
                        throttler=part_throttler)

# Extract
for corpus_name in ['Hardware']:
    corpus = get_ORM_instance(Corpus, session, corpus_name)
    print "Extracting Candidates from %s" % corpus
    %time candidates = ce.extract(\
        corpus.documents, corpus_name + ' Candidates', session)
    session.add(candidates)
    print "%s contains %d Candidates" % (candidates, len(candidates))
session.commit()

### View Results

In [ ]:
c = candidates[15]
print c.part
print c.part.char_start, c.part.char_end
print c.part.parent.page
print c.part.get_attrib_tokens('top')
print c.part.get_attrib_tokens('bottom')
print c.part.get_attrib_tokens('left')
print c.part.get_attrib_tokens('right')


from snorkel.entity_features import visual_binary_features
from snorkel.lf_helpers import get_aligned_lemmas, _bbox_from_span

print c.part.parent.document
for c in candidates:
    
    span1, span2 = c.get_arguments()
    
    feats = set()
    for f in visual_binary_features(span1, span2):
        feats.add(f)
    text1 =   span1.get_span()
    text2 = span2.get_span()
    if not  'Y_ALIGNED' in feats or span1.parent.page!=2: continue
    print '='*20
    print 'For candidate pair:'
    print span1.get_span()
    print span2.get_span()
    print 'Visual features are:'
    for f in feats: print f
    print 'LF is_aligned_with_lemmas:', 'min' in get_aligned_lemmas(span2)
    print 'Phrase1', span1.parent,  span1.parent.page#.text, span1.parent.bbox
    print 'Phrase2', span2.parent,  span2.parent.page#.text, span2.parent.bbox, span2.parent.page
    print _bbox_from_span(span1), _bbox_from_span(span2)
    
print len(candidates)

In [ ]:
context_parser.vizlink.display_links(20)

In [ ]:
# Note: Select image and press any key to close image.
context_parser.vizlink.display_candidates(candidates, page_num=2, display=False)

In [ ]:
# pdf_file = context_parser.vizlink.pdf_file

# context_parser.vizlink.display_word('BC546', page_num=2)
# context_parser.vizlink.display_candidates(candidates, page_num=2)

In [ ]:
# from visual_linking import display_boxes, get_box

# boxes = []
# for c in candidates:
#     boxes.append(get_box(c.part))
# boxes = list(set(boxes))
# display_boxes(pdf_file, boxes, page_num=2)

### Display Ordering of PDF Word List 

Display Ordering on a black page - Not very easy to display because of superposition

In [ ]:
# import numpy as np
# import cv2
# import math

# page_num = 2
# page_height = 792
# page_width = 612
# img = np.ones((page_height,page_width,3))*255
# font = cv2.FONT_HERSHEY_SIMPLEX
# letter_width = 3
# i = 0
# for word_id, _ in pdf_word_list:
#     if word_id[0] == page_num:
#         i += 1
#         _, top, left, bottom, right = coordinate_map[word_id]
#         cv2.rectangle(img, (left, top), (right, bottom), (0,255,0), 1)
#         cv2.putText(img, 
#                     str(i), 
#                     ((left + right)/2 - letter_width*int(math.ceil(math.log10(i))), 
#                     bottom + (top - bottom)/4), 
#                     font, 
#                     0.3, 
#                     (255,0,0), 
#                     1)
# cv2.imshow('PDF Word List Order',img)
# cv2.waitKey() # press any key to exit the opencv output 
# cv2.destroyAllWindows() 

# The end.